# Python Spark SQL Notebook

The final output gives for each parameter the best and worse county and its values, also gives all the counties if the value is the same, because I assumed a analysis on this dataset would be best if shown all the counties with worse and best value, and not just one for each instance.
This notebook exemplifies the execution of a Spark program in Python, using the SQL interface.
In this example, spark runs in standalone mode and reads data from the local filesystem, while in cluster mode data is read typically from HDFS dsitributed file system.

Spark documentation available at:
https://spark.apache.org/docs/2.3.1/


### Please first run this Download the dataset 

Dataset is being downloaded from a dropbox link
Contains a small subset of the original dataset
Dataset contains the information from air quality monitoring facilities across the U.S.A.

In [3]:
!wget -O epa_hap_daily_summary_small https://www.dropbox.com/s/4jxfdsgn2tdo7zo/epa_hap_daily_summary-small.csv?dl=0

--2021-12-23 18:33:53--  https://www.dropbox.com/s/4jxfdsgn2tdo7zo/epa_hap_daily_summary-small.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:6024:18::a27d:4412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.68.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/4jxfdsgn2tdo7zo/epa_hap_daily_summary-small.csv [following]
--2021-12-23 18:33:56--  https://www.dropbox.com/s/raw/4jxfdsgn2tdo7zo/epa_hap_daily_summary-small.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca67c4e5b8f5220e50ad5ea3522.dl.dropboxusercontent.com/cd/0/inline/BcZSQ_YIYVSI5ZlTzREl7SnyFkOckGZLefhtPtp0rlG_NAiJDUFp8CpoU9y2BATw8SKbsUtNjDKZC03bgTnCy5E08CPAaaZCCP_n6W1bdyONegadn642xGhO_jL2TpymkT4V3QgW5XI6RZDDj7cgBHIP/file# [following]
--2021-12-23 18:33:56--  https://uca67c4e5b8f5220e50ad5ea3522.dl.dropboxusercontent.com/cd/0/inline/BcZSQ_YIYVSI5ZlTzREl7SnyF

### Before doing anything in this notebook in order to answer some questions, we need to import specific libraries and initiate a spark session locally..
### Some initial exploratory analysis is required in order to better understand what data we are going to manipulate




In [74]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

spark = SparkSession.builder.master('local[*]').appName('Q2').getOrCreate()
sc = spark.sparkContext


In [75]:
lines = sc.textFile('epa_hap_daily_summary_small')

In [76]:
df=spark.read.option("header","true").csv(lines)

In [77]:
df.describe()

DataFrame[summary: string, state_code: string, county_code: string, site_num: string, parameter_code: string, poc: string, latitude: string, longitude: string, datum: string, parameter_name: string, sample_duration: string, pollutant_standard: string, date_local: string, units_of_measure: string, event_type: string, observation_count: string, observation_percent: string, arithmetic_mean: string, first_max_value: string, first_max_hour: string, aqi: string, method_code: string, method_name: string, local_site_name: string, address: string, state_name: string, county_name: string, city_name: string, cbsa_name: string, date_of_last_change: string]

### Answering Question 1 using SparkSQL, Which states have more/less monitors? (Rank states!) 

In [15]:
states_monitors = df = spark.read.csv(lines, inferSchema=True,header=True)

###### Registers this DataFrame as a temporary table using the given name.

###### The lifetime of this temporary table is tied to the SparkSession that was used to create this DataFrame.

In [30]:
states_monitors.registerTempTable("data")

###### After creating my temporary table I want to query it, to show only the distinct occurences of state and address, and 
###### and after that I want to group by state name column and count the number of occurences, sorting it in descending order, showing all 54 rows

In [63]:
state_address=spark.sql("select state_name, address from data").distinct().groupBy("state_name").count().sort("count", ascending=False).show(54)

+--------------------+-----+
|          state_name|count|
+--------------------+-----+
|          California|  161|
|               Texas|  132|
|           Minnesota|   94|
|                Ohio|   89|
|            Michigan|   83|
|            New York|   66|
|      South Carolina|   64|
|             Montana|   60|
|        Pennsylvania|   60|
|             Florida|   57|
|             Indiana|   52|
|            Colorado|   50|
|      North Carolina|   49|
|            Illinois|   49|
|          Washington|   42|
|           Louisiana|   40|
|             Arizona|   38|
|              Kansas|   37|
|             Georgia|   34|
|              Oregon|   31|
|            Kentucky|   30|
|             Alabama|   29|
|           Tennessee|   28|
|           Wisconsin|   24|
|          New Jersey|   23|
|               Maine|   21|
|             Vermont|   21|
|         Mississippi|   21|
|            Virginia|   20|
|            Oklahoma|   20|
|       Massachusetts|   19|
|             

#### Answering Question 2 using SparkSQL, Which counties have the best/worst air quality? (Rank counties considering pollutants’ level!)
#### In this question my approach was to do a window view and partition by the parameter code because I figured the most value would be taken from figuring out what are the worst and best counties considering the different pollutants, this way it would be possible to quickly figure out for a specific dangerous pollutant what are the counties that need the most interventnion

In [64]:
county_param_value = df = spark.read.csv(lines,inferSchema =True,header=True)

In [65]:
county_param_value_small = county_param_value.drop('state_code','county_code','site_num','poc','latitude','longitude','datum','parameter_name','sample_duration','pollutant_standard','date_local','units_of_measure','event_type','observation_count','observation_percent','first_max_value','first_max_hour','aqi','method_code','method_name','local_site_name','address','state_name','city_name','cbsa_name','date_of_last_change')


In [78]:
from pyspark.sql import functions as F
from pyspark.sql import Window as W

In [67]:
window_spec = W.partitionBy("parameter_code")

In [68]:
#I want to present the best and worst county for each pollutant, also showing the arithmetic mean value.

In [69]:
county_param_value_small.filter(county_param_value_small.arithmetic_mean != 0).withColumn("max_arithmetic_mean", F.max("arithmetic_mean").over(window_spec))\
  .withColumn("min_arithmetic_mean", F.min("arithmetic_mean").over(window_spec))\
  .filter((F.col("arithmetic_mean") == F.col("max_arithmetic_mean")) | (F.col("arithmetic_mean") == F.col("min_arithmetic_mean")))\
  .select("parameter_code", "county_name", "arithmetic_mean").distinct().show(15)
  #filter de zeros? maybe não quero isto porque não quero discartar o que têm valor 0 da minha análise...
  #df = df.filter(df.A != 0)

+--------------+---------------+---------------+
|parameter_code|    county_name|arithmetic_mean|
+--------------+---------------+---------------+
|         12103|          Roane|         1.0E-5|
|         12103|Lewis and Clark|            1.2|
|         43817|          Kings|          20.04|
|         43817|      San Diego|          0.004|
|         43817|          Marin|          0.004|
|         43218|         Denton|       0.004545|
|         43218|           Wise|       0.004545|
|         43218|       Imperial|          128.0|
|         43829|           Lake|          15.28|
|         43829|        Suffolk|          0.003|
|         82112|           Mesa|          107.4|
|         82112|        Fayette|           0.01|
|         82112|         Carter|           0.01|
|         43831|        Passaic|           0.01|
|         43831|          Anoka|           0.01|
+--------------+---------------+---------------+
only showing top 15 rows



In [31]:
sc.stop()


#### Answering Question 3 using SparkSQL, Which states have the best/worst air quality? (Rank counties considering pollutants’ level!)

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql import Window as W

In [84]:
state_param_value_small = df.drop('state_code','county_code','site_num','poc','latitude','longitude','datum','parameter_name','sample_duration','pollutant_standard','date_local','units_of_measure','event_type','observation_count','observation_percent','first_max_value','first_max_hour','aqi','method_code','method_name','local_site_name','address','county_name','city_name','cbsa_name','date_of_last_change')
# i just want these columns: 'parameter_code','state_name','arithmetic_mean' im not gonna use select 
#because i think it gives me an object noneType not a dataframe

In [85]:
window_spec = W.partitionBy("parameter_code")

In [86]:
state_param_value_small.filter(state_param_value_small.arithmetic_mean != 0).withColumn("max_arithmetic_mean", F.max("arithmetic_mean").over(window_spec))\
  .withColumn("min_arithmetic_mean", F.min("arithmetic_mean").over(window_spec))\
  .filter((F.col("arithmetic_mean") == F.col("max_arithmetic_mean")) | (F.col("arithmetic_mean") == F.col("min_arithmetic_mean")))\
  .select("parameter_code", "state_name", "arithmetic_mean").distinct().show(15)
  #filter de zeros? maybe não quero isto porque não quero discartar o que têm valor 0 da minha análise...
  #df = df.filter(df.A != 0)

+--------------+------------+---------------+
|parameter_code|  state_name|arithmetic_mean|
+--------------+------------+---------------+
|         43831|  California|            3.3|
|         43831|  California|           1.23|
|         43509|     Indiana|              1|
|         43509|     Indiana|            7.5|
|         43509|   Wisconsin|              1|
|         82128|      Kansas|          2.647|
|         82128|     Montana|           1.22|
|         82132|    Michigan|              1|
|         82132|     Florida|              1|
|         82132|  California|              1|
|         82132|      Kansas|              1|
|         82132|    New York|              1|
|         82132|Rhode Island|              1|
|         82132|   Minnesota|              1|
|         82132|    Kentucky|              1|
+--------------+------------+---------------+
only showing top 15 rows



In [73]:
spark.stop()